In [1]:
import pandas as pd
import re
import json
import time
from pandas.io.json import json_normalize
import plotly.plotly as py
from plotly import tools
import plotly.graph_objs as go

In [2]:
url = r'D:\DATA\python files\kaggle_kernel\google_revenue_prediction\train_v2.csv'
url = re.sub(r'\\',r'/',url)

In [3]:
def single2double(x):
    x = re.sub(r'\'',r'"',x)
    json.loads(x)
    x = re.sub(r'"',r'',x)
    return x

In [4]:
def load_df(csv_path= url , nrows=None):
    starttime = time.time()
    chunks = []
    loop = True
    chunk_size = 1000
    
    columns_use = ['channelGrouping',
                    'customDimensions',
                    'date',
                    'device', 
                    'fullVisitorId',
                    'geoNetwork',
                    'socialEngagementType',
                    'totals',
                    'trafficSource',
                    'visitId',
                    'visitNumber',
                    'visitStartTime']
    data = pd.read_csv(csv_path,
                       converters={'customDimensions':single2double,
                                   'device':json.loads,
                                   'geoNetwork':json.loads,
                                   'totals':json.loads,
                                   'trafficSource':json.loads},
                       dtype={'fullVisitorId': 'str'},                                    
                       usecols = columns_use,
                       iterator =True)

                      
    while loop:
        try:
            chunk = data.get_chunk(chunk_size)
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print('Iteration is stopped')   
    sample = pd.concat(chunks,axis = 0)

    time_use = time.time() - starttime
    print(f'use {time_use}s')
    return sample

In [5]:
train = load_df()

Iteration is stopped
use 154.50386214256287s


## 用json_normalize将json格式的列分开


In [6]:
def json_split(df):
    JSON_COLUMNS = ['device','geoNetwork','totals','trafficSource']
    for column in JSON_COLUMNS:
        columns_as_df = json_normalize(df[column].values.tolist())
        columns_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in columns_as_df.columns]
        df = df.drop(column, axis=1).merge(columns_as_df, right_index=True, left_index=True)
    return df

In [7]:
train = json_split(train)
train.shape

(1708337, 59)

## 将customDimensions列split，因为该列是字符串

In [8]:
def customer(df):
    starttime = time.time()
    df.loc[df['customDimensions'] == '[]','customDimensions'] = '[{index: notknow, value: notknow}]'
    df['customDimensions'] = df['customDimensions'].apply(lambda x: re.findall(r'(?<={).+(?=})',x)[0])
    column_cus = df['customDimensions'].str.split(',',expand = True)
    column_cus = column_cus.applymap(lambda x:re.findall(r'(?<=\: ).+',x)[0])
    column_cus.rename(columns = {0:'customDimensions_index',
                                 1:'customDimensions_value'},
                      inplace = True)
    df = df.drop('customDimensions',axis = 1).merge(column_cus,
                                                    right_index = True,
                                                    left_index = True)
    time_use = time.time() - starttime
    print('use %ss'%time_use)
    return df

In [9]:
train = customer(train)

use 21.762195110321045s


## 删掉常数列

In [10]:
def deleteconstant(df):
    constantcol = [col for col in df.columns if df[col].nunique(dropna = False) == 1]
    df.drop(constantcol,axis = 1,inplace = True)
    print(f'the name of the features {constantcol}')
    return df

In [11]:
train = deleteconstant(train)
train.shape

the name of the features ['socialEngagementType', 'device.browserSize', 'device.browserVersion', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits', 'trafficSource.adwordsClickInfo.criteriaParameters']


(1708337, 41)

In [12]:
train['totals.transactionRevenue'] = train['totals.transactionRevenue'].astype('float')

## 观察数据

In [13]:
tools.set_credentials_file(username='aptx4869', api_key='NngEVVmuAKzvMUSgAu4C')

In [14]:
def horizontal_bar_chart(df,color):
    trace = go.Bar(x = df.values[::-1],
                   y = df.index[::-1],
                   showlegend = False,
                   orientation = 'h',
                   marker = dict(color = color,
                                 line = dict(color = 'rgba(58, 71, 80, 1.0)'))
                   )
    return trace

In [15]:
device_columns = ['device.browser','device.deviceCategory','device.operatingSystem']
colors = [['rgba(50, 171, 96, 0.6)']*3,['rgba(71, 58, 131, 0.8)']*3,['rgba(246, 78, 139, 0.6)']*3]
traces = []

In [16]:
for devicecol,barcolor in zip(device_columns,colors):
    cnt_srs = train.groupby(devicecol)['totals.transactionRevenue'].agg(['size', 'count', 'mean'])
    cnt_srs = cnt_srs.sort_values(by = 'count',ascending = False)
    traces = traces +[horizontal_bar_chart(cnt_srs[column].head(5),color) for column,color in zip(cnt_srs.columns,barcolor)]


In [17]:
fig = tools.make_subplots(rows = 3,
                          cols =3,
                          subplot_titles=["Device Browser - Count", "Device Browser - Non-zero Revenue Count", "Device Browser - Mean Revenue",
                                          "Device Category - Count",  "Device Category - Non-zero Revenue Count", "Device Category - Mean Revenue", 
                                          "Device OS - Count", "Device OS - Non-zero Revenue Count", "Device OS - Mean Revenue"])

fig.append_trace(traces[0],1,1)
fig.append_trace(traces[1],1,2)
fig.append_trace(traces[2],1,3)
fig.append_trace(traces[3],2,1)
fig.append_trace(traces[4],2,2)
fig.append_trace(traces[5],2,3)
fig.append_trace(traces[6],3,1)
fig.append_trace(traces[7],3,2)
fig.append_trace(traces[8],3,3)

fig['layout'].update(height = 1200,width = 1200,title = 'Device plots')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



In [18]:
py.iplot(fig, filename='device-plots')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1708337 entries, 0 to 1708336
Data columns (total 41 columns):
channelGrouping                                 object
date                                            int64
fullVisitorId                                   object
visitId                                         int64
visitNumber                                     int64
visitStartTime                                  int64
device.browser                                  object
device.deviceCategory                           object
device.isMobile                                 bool
device.operatingSystem                          object
geoNetwork.city                                 object
geoNetwork.continent                            object
geoNetwork.country                              object
geoNetwork.metro                                object
geoNetwork.networkDomain                        object
geoNetwork.region                               object
geoNetwork.su